In [ ]:
pip install -U numpy==1.11.0

In [2]:
import numpy as np
import pandas as pd

Read the dataset from where it is downloaded in the system. It consists of two files ‘ratings.dat’ and ‘movies.dat’ which need to be read.

In [3]:
data = pd.io.parsers.read_csv('/content/drive/MyDrive/Movie_Recommender_System/ratings.dat', 
    names=['user_id', 'movie_id', 'rating', 'time'],
    engine='python', delimiter='::')

movie_data = pd.io.parsers.read_csv('/content/drive/MyDrive/Movie_Recommender_System/movies.dat',
    names=['movie_id', 'title', 'genre'],
    engine='python', delimiter='::')

Create the rating matrix with rows as movies and columns as users.

In [4]:
ratings_mat = np.ndarray( shape=(np.max(data.movie_id.values), np.max(data.user_id.values)), dtype=np.uint8)
ratings_mat[data.movie_id.values-1, data.user_id.values-1] = data.rating.values

Normalise the matrix.

In [5]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

Compute the Singular Value Decomposition (SVD).

In [6]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

Define a function to calculate the cosine similarity. Sort by most similar and return the top N results.

In [7]:
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

Define a function to print top N similar movies.

In [8]:
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

Initialise the value of k principal components, id of the movie as given in the dataset, and number of top elements to be printed.

In [19]:
k = 50
movie_id = 23 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


Print the top N similar movies.

In [20]:
print_similar_movies(movie_data, movie_id, indexes)

Recommendations for Assassins (1995): 

Assassins (1995)
Eye for an Eye (1996)
Jury Duty (1995)
Chungking Express (1994)
Kids of the Round Table (1995)
It Takes Two (1995)
Unzipped (1995)
Screamers (1995)
Home for the Holidays (1995)
Crossing Guard, The (1995)
